<a href="https://colab.research.google.com/github/vimesh630/Spice_Price_Predction_VERGER/blob/Cinnamon/Preprocessing_for_Cinnamon_Price_Forecasting_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Mount Google Drive

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#2. Import Libraries

In [14]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import KNNImputer
from datetime import datetime

#3. Set Paths

In [15]:
dataset_path = '/content/drive/MyDrive/VERGER/Cinnamon_Price_Prediction/Cinnamon_Dataset_New_0001.xlsx'
save_path = '/content/drive/MyDrive/VERGER/Cinnamon_Price_Prediction'
os.makedirs(save_path, exist_ok=True)

#4. Load Dataset

In [16]:
df = pd.read_excel(dataset_path)
print("Initial shape:", df.shape)
df.head()

Initial shape: (2772, 16)


Month Grade     Region  ...  Exchange_Rate  Inflation_Rate  Fuel_Price
0 2020-01-01  alba  ratnapura  ...       181.4036             5.7         104
1 2020-01-03  alba    colombo  ...       181.4036             5.7         104
2 2020-01-04  alba   kalutara  ...       181.4036             5.7         104
3 2020-01-05  alba      galle  ...       181.4036             5.7         104
4 2020-01-06  alba     matara  ...       181.4036             5.7         104

[5 rows x 16 columns]

#5. Parse Dates

In [17]:
df['Month'] = pd.to_datetime(df['Month'], format='%Y-%m')
df = df.sort_values(by='Month')

#6. Handle Missing Values Using KNN Imputation

In [19]:
features_to_impute = df.columns.drop(['Month', 'Grade', 'Region'])
imputer = KNNImputer(n_neighbors=3)
df_imputed = pd.DataFrame(imputer.fit_transform(df[features_to_impute]), columns=features_to_impute)
df_imputed['Month'] = df.index.values

#7. Normalize the features

In [22]:
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df_imputed.drop('Month', axis=1))
scaled_df = pd.DataFrame(scaled_data, columns=features_to_impute)
scaled_df['Month'] = df_imputed['Month'].values

#8. Sequence Preparation

In [23]:
def create_seq2seq_data(df, sequence_length=12, prediction_length=3):
    X, y = [], []
    df = df.sort_values("Month").reset_index(drop=True)
    data = df.drop("Month", axis=1).values

    for i in range(len(data) - sequence_length - prediction_length + 1):
        X.append(data[i:i+sequence_length])
        y.append(data[i+sequence_length:i+sequence_length+prediction_length])

    return np.array(X), np.array(y)